In [1]:
import py_entitymatching as em
import pandas as pd
import numpy as np

A = em.read_csv_metadata('/Users/daravinds/Documents/Projects/CS-838/Stage4-IntegrationAndAnalysis/CS-838-bookdepository.csv',key='id')
B = em.read_csv_metadata('/Users/daravinds/Documents/Projects/CS-838/Stage4-IntegrationAndAnalysis/CS-838-goodreads.csv',key='id') 

# A = A.replace(np.nan, '', regex=True)
# B = B.replace(np.nan, '', regex=True)
A['Language'] = A['Language'].fillna('')
B['Language'] = B['Language'].fillna('')

A['Publishing Date'] = A['Publishing Date'].fillna('')
B['Publishing Date'] = B['Publishing Date'].fillna('')

A['Publisher'] = A['Publisher'].fillna('')
B['Publisher'] = B['Publisher'].fillna('')

A['Genres'] = A['Genres'].fillna('')
B['Genres'] = B['Genres'].fillna('')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [2]:
I = em.read_csv_metadata("train_set.csv", 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')

Metadata file is not present in the given path; proceeding to read the csv file.


In [3]:
# Generate a set of features
F = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

In [4]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=F, 
                            attrs_after=['match'])

# Impute feature vectors with the mean of the column values.
H = em.impute_table(H, 
                exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'],
                strategy='mean')

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


In [5]:
# Create a set of ML-matchers
lg = em.LogRegMatcher(name='LogReg', random_state=0)

# Train using feature vectors from I 
lg.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'], 
       target_attr='match')


In [6]:
J = em.read_csv_metadata("after_blocking_tuples.csv", 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')

# Convert J into a set of feature vectors using F
L = em.extract_feature_vecs(J, feature_table=F, show_progress=False)
L = L.fillna(0)
em.set_property(L, 'key', '_id')

Metadata file is not present in the given path; proceeding to read the csv file.


True

In [7]:
# Predict on L 
predictions = lg.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_id'], 
              append=True, target_attr='predicted', inplace=False, return_probs=True,
                        probs_attr='proba')

In [8]:
np.count_nonzero(predictions['predicted'])

879

In [9]:
df = pd.DataFrame(predictions)
matches_df = df.loc[df['predicted'] == 1]

match_map = dict()
for index, match in matches_df.iterrows():
    if match['ltable_id'] in match_map:
        existing_matches = match_map.get(match['ltable_id'])
    else:
        existing_matches = set()
    existing_matches.add(match['rtable_id'])
    match_map[match['ltable_id']] = existing_matches


In [10]:
all_values = set()
for values in match_map.values():
    for value in values:
        all_values.add(value)

A_ids = set(A['id'])
A_matched_ids = set(match_map.keys())
ltable_ids_without_match = A_ids.difference(A_matched_ids)

B_ids = set(B['id'])
B_matched_ids = all_values
rtable_ids_without_match = B_ids.difference(B_matched_ids)

merged_data = pd.DataFrame()
data_from_A = pd.DataFrame(A.loc[A['id'].isin(ltable_ids_without_match)], columns=["Title", "Authors", "Genres", "Publishing Date", "Pages", "Publisher", "Language"])
data_from_B = pd.DataFrame(B.loc[B['id'].isin(rtable_ids_without_match)], columns=["Title", "Authors", "Genres", "Publishing Date", "Pages", "Publisher", "Language"])
merged_data = data_from_A.append(data_from_B, ignore_index = True)

In [11]:
def merge_titles(records):
    length = 0
    for record in records:
        cur_title = record['Title'].strip()
        if len(cur_title) > length:
            length = len(cur_title)
            title = cur_title
    return title

In [12]:
def merge_authors(records):
    authors = list()
    length = 0
    for record in records:
        current_authors = record['Authors'].strip().split(":")
        if len(current_authors) > length:
            length = len(current_authors)
            authors = ":".join(current_authors)
    return authors

In [13]:
def merge_genres(records):
    genres = set(records[-1]['Genres'].strip().split(":"))
    for i in range(len(records) - 1):
        cur_genres = records[i]['Genres'].strip().split(":")
        genres.union(cur_genres)

    return ":".join(list(genres))

In [14]:
def merge_publishing_dates(records):
    for record in records:
        pub_date = record['Publishing Date'].strip()
        if pub_date is not None and len(pub_date) > 0:
            return pub_date

In [15]:
def merge_pages(records):
    for record in records:
        pages = record['Pages'].strip()
        if pages is not None and len(pages) > 0:
            return pages

In [16]:
def merge_publishers(records):
    for record in records:
        publishers = record['Publisher'].strip()
        if publishers is not None and len(publishers) > 0:
            return publishers

In [17]:
def merge_languages(records):
    for record in records:
        languages = record['Language'].strip()
        if languages is not None and len(languages) > 0:
            return languages

In [18]:
def merge_tuples(records):
    final_title = merge_titles(records)
    authors = merge_authors(records)
    genres = merge_genres(records)
    publishing_date = merge_publishing_dates(records)
    pages = merge_pages(records)
    publisher = merge_publishers(records)
    language = merge_languages(records)
    final_record = (final_title, authors, genres, publishing_date, pages, publisher, language)

    return final_record

In [21]:
match_map

final_records = list()
for l_id, r_ids in match_map.items():
    records = list()
    matches= B.loc[B['id'].isin(r_ids)]
    for index, match in matches.iterrows():
        records.append(match)
    matches = A.loc[A['id'] == l_id]
    for index, match in matches.iterrows():
        records.append(match)

    record = merge_tuples(records)
    final_records.append(record)
    
final_df = pd.DataFrame(final_records, columns=["Title", "Authors", "Genres", "Publishing Date", "Pages", "Publisher", "Language"])
merged_data2 = merged_data.append(final_df, ignore_index = True)
merged_data2.to_csv("unified_table.csv", index=False)

,Title,Authors,Genres,Publishing Date,Pages,Publisher,Language
0,Fifty Shades of Grey : Book One of the Fifty Shades Trilogy (Fifty Shades of Grey Series),E.L. James,Romance Books:Romance,May 25th 2011,356,Vintage,English
1,Fifty Shades Freed : Book Three of the Fifty Shades Trilogy (Fifty Shades of Grey Series),E.L. James,Romance Books:Adult & Contemporary Romance:Romance,May 25th 2011,356,Vintage,English
2,Fifty Shades Darker : Book Two of the Fifty Shades Trilogy (Fifty Shades of Grey Series),E.L. James,Romance Books:Adult & Contemporary Romance:Contemporary Fiction:Romance,May 25th 2011,356,Vintage,English
3,Fifty Shades of Grey,E.L. James,Adult & Contemporary Romance:Contemporary Fiction:Thriller Books:Thrillers,May 25th 2011,356,Vintage,English
4,Fifty Shades Trilogy : Fifty Shades of Grey Fifty Shades Darker Fifty Shades Freed 3-Volume Boxe...,E.L. James,Erotic Fiction:Contemporary Fiction:Adult & Contemporary Romance,May 25th 2011,356,Vintage,English
5,Fifty Shades of Grey : (Movie tie-in edition): Book one of the Fifty Shades Series,E.L. James,Romance Books:Adult & Contemporary Romance:Contemporary Fiction:Romance,May 25th 2011,356,Vintage,English
6,Grey : Fifty Shades of Grey as Told by Christian (US edition),E.L. James,Fiction Books:Erotic Fiction,May 25th 2011,356,Vintage,English
7,Large Print : Fifty Shades Of Grey,E.L. James,Contemporary Fiction:Adult & Contemporary Romance:Romance Books:Erotic Fiction:Romance,May 25th 2011,356,Vintage,English
8,Fifty Shades of Grey,E.L. James,Erotic Fiction:Contemporary Fiction:Adult & Contemporary Romance,May 25th 2011,356,Vintage,English
9,Fifty Shades of Grey (Movie Tie-In Edition) : Book One of the Fifty Shades Trilogy,E.L. James,Contemporary Fiction:Adult & Contemporary Romance:Romance Books:Erotic Fiction:Romance,May 25th 2011,356,Vintage,English


In [20]:
# len(merged_data2)

7098